In [59]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [60]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [61]:
import pandas as pd
Data =  pd.read_json('/content/drive/My Drive/Sarcasm Detection/Arabic News Headlines Dataset.json', orient = 'split')
Data.head()

,headlines,topic,length,is_sarcastic
0,طالب يصبح متعلّماً بعد دراسته وحدة عن القائد ف...,Society,12,1
1,سموحة يقلص الفارق ويسجل هدفا ثانيا في شباك الم...,Sports,11,0
2,ليتها الحدود – الولايات المتحدة تهدد بفرض عقوب...,Misc,17,1
3,وزيرة الهجرة تتواصل مع سفير مصر في نيوزيلندا ل...,None,14,0
4,ليتها الحدود – مصر تقرر إرسال أعضاء السائح الب...,Misc,12,1


In [0]:
X = Data['headlines']
Labels = Data['is_sarcastic']

In [63]:
import numpy as np
max_sequence_length = np.max(Data['length'])
print("max length is:", max_sequence_length)

max length is: 34


In [64]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))

Found 20595 unique tokens.


In [65]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (6669, 34)


In [66]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(Labels))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (6669, 2)


In [0]:
import pickle as p
fastText = p.load(open('/content/drive/My Drive/Sarcasm Detection/FastText_Wiki_Embeddings.p', 'rb'))

In [68]:
# prepare embedding matrix  (matrix of embeddings of the words in our dataset)
num_words = len(word_index) + 1
# initialize the np array
unKnown = list()
embedding_matrix = np.zeros((num_words, 300))
l=0
# find the embeddings from the pre trained embedding matrix of fast text wiki
for word, i in word_index.items():
    embedding_vector = fastText.get(word)
    if embedding_vector is not None:
        l+=1
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
      unKnown.append(word)
print("the number of words found in fast text embeddings =", l)
print("the number of words not found in fast text embeddings =", 20595- l)

the number of words found in fast text embeddings = 17289
the number of words not found in fast text embeddings = 3306


In [0]:
from keras.initializers import Constant
from keras.layers import Input, Embedding, Conv2D, GlobalMaxPooling2D, Dense
from keras.models import Model

In [0]:
# shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


## split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [0]:
embedding_layer = Embedding(num_words,
                            300,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

In [0]:
rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9
                                   , epsilon=None, decay=0.3)



In [0]:
# the model

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')             ## input layer
embedded_sequences = embedding_layer(sequence_input)                            ## embeddings layer
embedded_sequences = keras.layers.core.Reshape((max_sequence_length,300,1))(embedded_sequences)  ## reshape layer
x1 = Conv2D(2, (2, 300), activation='relu')(embedded_sequences)                 ## conv 1
x1 = GlobalMaxPooling2D()(x1)                                                   ## max 1
x2 = Conv2D(2, (3, 300), activation='relu')(embedded_sequences)                 ## conv 2
x2 = GlobalMaxPooling2D()(x2)                                                   ## max 2
x3 = Conv2D(2, (4, 300), activation='relu')(embedded_sequences)                 ## conv 3
x3 = GlobalMaxPooling2D()(x3)                                                   ## max 3
x  = keras.layers.concatenate([x1, x2, x3])                                     ## concatenate
preds = Dense(2, activation='softmax')(x)                                       ## softmax layer

model = Model(sequence_input, preds)

In [94]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = rmsprop,
              metrics = ['acc'])

model.fit(x_train, y_train, 
          validation_split = 0.2, 
          batch_size = 32, 
          epochs = 200)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/200
4001/4001 [==============================] - 2s 487us/step - loss: 0.6181 - acc: 0.7401 - val_loss: 0.5934 - val_acc: 0.7822
Epoch 2/200
4001/4001 [==============================] - 1s 155us/step - loss: 0.5869 - acc: 0.7878 - val_loss: 0.5811 - val_acc: 0.7942
Epoch 3/200
4001/4001 [==============================] - 1s 160us/step - loss: 0.5772 - acc: 0.8008 - val_loss: 0.5742 - val_acc: 0.7982
Epoch 4/200
4001/4001 [==============================] - 1s 158us/step - loss: 0.5711 - acc: 0.8063 - val_loss: 0.5693 - val_acc: 0.8042
Epoch 5/200
4001/4001 [==============================] - 1s 153us/step - loss: 0.5667 - acc: 0.8110 - val_loss: 0.5657 - val_acc: 0.8062
Epoch 6/200
4001/4001 [==============================] - 1s 160us/step - loss: 0.5633 - acc: 0.8128 - val_loss: 0.5627 - val_acc: 0.8082
Epoch 7/200
4001/4001 [==============================] - 1s 137us/step - loss: 0.5605 - acc: 0.8150 - val_loss: 0.5602 - val_acc: 

In [0]:
from keras import backend as K

def recall(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall


      
def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
      

def f1(y_true, y_pred):
    
    Precision = precision(y_true, y_pred)
    Recall = recall(y_true, y_pred)
    f1 = 2*((Precision*Recall)/(Precision+Recall+K.epsilon()))
    
    return f1



In [108]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = rmsprop,
              metrics = [precision])
model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 200)

loss, prec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', prec)


Train on 4001 samples, validate on 1001 samples
Epoch 1/200
4001/4001 [==============================] - 2s 537us/step - loss: 0.4677 - precision: 0.8493 - val_loss: 0.4762 - val_precision: 0.8272
Epoch 2/200
4001/4001 [==============================] - 1s 140us/step - loss: 0.4677 - precision: 0.8493 - val_loss: 0.4762 - val_precision: 0.8272
Epoch 3/200
4001/4001 [==============================] - 1s 141us/step - loss: 0.4676 - precision: 0.8493 - val_loss: 0.4762 - val_precision: 0.8272
Epoch 4/200
4001/4001 [==============================] - 1s 137us/step - loss: 0.4676 - precision: 0.8493 - val_loss: 0.4761 - val_precision: 0.8272
Epoch 5/200
4001/4001 [==============================] - 1s 139us/step - loss: 0.4676 - precision: 0.8493 - val_loss: 0.4761 - val_precision: 0.8272
Epoch 6/200
4001/4001 [==============================] - 1s 138us/step - loss: 0.4675 - precision: 0.8493 - val_loss: 0.4761 - val_precision: 0.8272
Epoch 7/200
4001/4001 [==============================] - 1

In [109]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = rmsprop,
              metrics = [recall])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 200)

loss, rec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', rec)

Train on 4001 samples, validate on 1001 samples
Epoch 1/200
4001/4001 [==============================] - 2s 539us/step - loss: 0.4614 - recall: 0.8503 - val_loss: 0.4702 - val_recall: 0.8322
Epoch 2/200
4001/4001 [==============================] - 1s 141us/step - loss: 0.4614 - recall: 0.8503 - val_loss: 0.4702 - val_recall: 0.8322
Epoch 3/200
4001/4001 [==============================] - 1s 139us/step - loss: 0.4613 - recall: 0.8503 - val_loss: 0.4701 - val_recall: 0.8322
Epoch 4/200
4001/4001 [==============================] - 1s 141us/step - loss: 0.4613 - recall: 0.8503 - val_loss: 0.4701 - val_recall: 0.8322
Epoch 5/200
4001/4001 [==============================] - 1s 143us/step - loss: 0.4613 - recall: 0.8503 - val_loss: 0.4701 - val_recall: 0.8322
Epoch 6/200
4001/4001 [==============================] - 1s 137us/step - loss: 0.4613 - recall: 0.8505 - val_loss: 0.4701 - val_recall: 0.8322
Epoch 7/200
4001/4001 [==============================] - 1s 140us/step - loss: 0.4612 - recall

In [110]:
model.compile(loss = 'categorical_crossentropy',
          optimizer = rmsprop, 
          metrics = [f1])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 200)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/200
4001/4001 [==============================] - 2s 561us/step - loss: 0.4571 - f1: 0.8515 - val_loss: 0.4660 - val_f1: 0.8322
Epoch 2/200
4001/4001 [==============================] - 1s 145us/step - loss: 0.4571 - f1: 0.8515 - val_loss: 0.4660 - val_f1: 0.8322
Epoch 3/200
4001/4001 [==============================] - 1s 142us/step - loss: 0.4570 - f1: 0.8515 - val_loss: 0.4660 - val_f1: 0.8322
Epoch 4/200
4001/4001 [==============================] - 1s 144us/step - loss: 0.4570 - f1: 0.8515 - val_loss: 0.4660 - val_f1: 0.8322
Epoch 5/200
4001/4001 [==============================] - 1s 141us/step - loss: 0.4570 - f1: 0.8515 - val_loss: 0.4660 - val_f1: 0.8322
Epoch 6/200
4001/4001 [==============================] - 1s 143us/step - loss: 0.4570 - f1: 0.8515 - val_loss: 0.4659 - val_f1: 0.8322
Epoch 7/200
4001/4001 [==============================] - 1s 139us/step - loss: 0.4570 - f1: 0.8515 - val_loss: 0.4659 - val_f1: 0.8322
Epoch 8

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 34, 300)      6178800     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 34, 300, 1)   0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 33, 1, 2)     1202        reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.2, amsgrad=False)


In [134]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = ['acc'])

model.fit(x_train, y_train, 
          validation_split = 0.2, 
          batch_size = 32, 
          epochs = 200)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/200
4001/4001 [==============================] - 3s 837us/step - loss: 0.6200 - acc: 0.7216 - val_loss: 0.5993 - val_acc: 0.7602
Epoch 2/200
4001/4001 [==============================] - 1s 182us/step - loss: 0.5815 - acc: 0.7906 - val_loss: 0.5830 - val_acc: 0.7782
Epoch 3/200
4001/4001 [==============================] - 1s 173us/step - loss: 0.5683 - acc: 0.8038 - val_loss: 0.5737 - val_acc: 0.7842
Epoch 4/200
4001/4001 [==============================] - 1s 179us/step - loss: 0.5600 - acc: 0.8100 - val_loss: 0.5673 - val_acc: 0.7902
Epoch 5/200
4001/4001 [==============================] - 1s 180us/step - loss: 0.5540 - acc: 0.8133 - val_loss: 0.5623 - val_acc: 0.7932
Epoch 6/200
4001/4001 [==============================] - 1s 180us/step - loss: 0.5494 - acc: 0.8148 - val_loss: 0.5584 - val_acc: 0.7972
Epoch 7/200
4001/4001 [==============================] - 1s 175us/step - loss: 0.5456 - acc: 0.8170 - val_loss: 0.5551 - val_acc: 

In [135]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [precision])
model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 200)

loss, prec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', prec)


Train on 4001 samples, validate on 1001 samples
Epoch 1/200
4001/4001 [==============================] - 3s 798us/step - loss: 0.4783 - precision: 0.8433 - val_loss: 0.4927 - val_precision: 0.8272
Epoch 2/200
4001/4001 [==============================] - 1s 155us/step - loss: 0.4780 - precision: 0.8435 - val_loss: 0.4925 - val_precision: 0.8272
Epoch 3/200
4001/4001 [==============================] - 1s 157us/step - loss: 0.4779 - precision: 0.8438 - val_loss: 0.4923 - val_precision: 0.8272
Epoch 4/200
4001/4001 [==============================] - 1s 158us/step - loss: 0.4777 - precision: 0.8438 - val_loss: 0.4922 - val_precision: 0.8272
Epoch 5/200
4001/4001 [==============================] - 1s 157us/step - loss: 0.4776 - precision: 0.8440 - val_loss: 0.4920 - val_precision: 0.8272
Epoch 6/200
4001/4001 [==============================] - 1s 159us/step - loss: 0.4774 - precision: 0.8438 - val_loss: 0.4919 - val_precision: 0.8272
Epoch 7/200
4001/4001 [==============================] - 1

In [136]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [recall])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 200)

loss, rec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', rec)

Train on 4001 samples, validate on 1001 samples
Epoch 1/200
4001/4001 [==============================] - 3s 810us/step - loss: 0.4661 - recall: 0.8495 - val_loss: 0.4812 - val_recall: 0.8282
Epoch 2/200
4001/4001 [==============================] - 1s 158us/step - loss: 0.4660 - recall: 0.8495 - val_loss: 0.4811 - val_recall: 0.8282
Epoch 3/200
4001/4001 [==============================] - 1s 159us/step - loss: 0.4659 - recall: 0.8495 - val_loss: 0.4811 - val_recall: 0.8282
Epoch 4/200
4001/4001 [==============================] - 1s 161us/step - loss: 0.4658 - recall: 0.8495 - val_loss: 0.4810 - val_recall: 0.8282
Epoch 5/200
4001/4001 [==============================] - 1s 156us/step - loss: 0.4658 - recall: 0.8495 - val_loss: 0.4809 - val_recall: 0.8292
Epoch 6/200
4001/4001 [==============================] - 1s 162us/step - loss: 0.4657 - recall: 0.8495 - val_loss: 0.4809 - val_recall: 0.8292
Epoch 7/200
4001/4001 [==============================] - 1s 160us/step - loss: 0.4657 - recall

In [137]:
model.compile(loss = 'categorical_crossentropy',
          optimizer = adam, 
          metrics = [f1])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 200)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/200
4001/4001 [==============================] - 3s 842us/step - loss: 0.4595 - f1: 0.8515 - val_loss: 0.4750 - val_f1: 0.8292
Epoch 2/200
4001/4001 [==============================] - 1s 155us/step - loss: 0.4594 - f1: 0.8513 - val_loss: 0.4749 - val_f1: 0.8292
Epoch 3/200
4001/4001 [==============================] - 1s 160us/step - loss: 0.4593 - f1: 0.8513 - val_loss: 0.4748 - val_f1: 0.8292
Epoch 4/200
4001/4001 [==============================] - 1s 175us/step - loss: 0.4593 - f1: 0.8513 - val_loss: 0.4748 - val_f1: 0.8292
Epoch 5/200
4001/4001 [==============================] - 1s 187us/step - loss: 0.4592 - f1: 0.8513 - val_loss: 0.4748 - val_f1: 0.8292
Epoch 6/200
4001/4001 [==============================] - 1s 187us/step - loss: 0.4592 - f1: 0.8513 - val_loss: 0.4747 - val_f1: 0.8292
Epoch 7/200
4001/4001 [==============================] - 1s 188us/step - loss: 0.4591 - f1: 0.8513 - val_loss: 0.4747 - val_f1: 0.8292
Epoch 8

In [0]:
model.save_weights('/content/drive/My Drive/Sarcasm Detection/Plots/CNN_optimized_Weights.h5')